#  Query Gaia for WDS entries - parallelized for multiprocessing across multiple cores
#### Summer 2022 -> revised in Spring 2023
#### This code should just be run in order and it will do its thing
#### Daphne Zakarian

In [ ]:
from astropy.io import ascii
from astropy.table import vstack, Table, unique
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy import table, log
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord, Distance, Angle
from astropy.time import Time
from astroquery.gaia import Gaia
from astroquery.utils.tap.model import job
from itertools import combinations
import multiprocessing
from multiprocessing import Queue, Pool, freeze_support, Process
import os
from IPython.display import display
from multiprocessing import set_start_method

## query_gaia(coordinate, radius)
 pretty self-explanatory... this makes the gaia query for each coordinate in the WDS, and searches for all objects (that fit the parallax and parallax error criteria)


In [ ]:
def query_gaia(coordinate, radius):

    # these column names list the info to pull from Gaia
    # if you change this, make sure to change the wds_to_gaia_query() function
    # to update that info in the tables themselves!!
    column_names = ['source_id', 'ref_epoch', 'ra', 'ra_error', 'dec',
        'dec_error', 'parallax', 'parallax_error', 'parallax_over_error','pmra',
        'pmra_error', 'pmdec', 'pmdec_error',
        'radial_velocity', 'radial_velocity_error',
        'astrometric_params_solved', 'visibility_periods_used',
        'astrometric_sigma5d_max','ruwe',
        'phot_g_mean_mag', 'phot_g_mean_flux_over_error',
        'phot_bp_mean_mag', 'phot_bp_mean_flux_over_error',
        'phot_rp_mean_mag', 'phot_rp_mean_flux_over_error',
        'bp_rp','phot_bp_rp_excess_factor']

    # the columns have to be a string, not a list
    # this turns the column list into a string for the query
    columns = ''
    for column in column_names:
        columns += column + ', '
    columns =  columns.rstrip(columns[-4])
    columns = columns[:len(columns)-2]
    columns

    # get the degree value for coordinate and radius
    ra = coordinate.ra.deg
    dec = coordinate.dec.deg
    radius = float(radius.to_value(u.deg))

    # query base:
    query_base = """
    SELECT {columns}
    FROM gaiadr3.gaia_source
    WHERE parallax > 1
    AND parallax_over_error > 5
    AND parallax_error < 2
    AND 1 = CONTAINS(
    POINT({ra}, {dec}),
    CIRCLE(ra, dec, {rad}))

    """



    # format the query with our specific info
    query = query_base.format(columns=columns, ra=ra, dec=dec, rad=radius)

    # make the query to gaia and save the results into astropy table
    job = Gaia.launch_job_async(query)
    job
    results = job.get_results()
    return results



## test queries for individual rows

In [ ]:
# # Read in the WDS table
# # vayu's lab comp
# # path = 'C:/Users/sc36/Documents/DaphneUSNO/NOFS copy-20230218T215456Z-001/NOFS copy/wdstab6-27.ecsv'

# # wiser's lab comp
# path = '/home/student/djz7128/djz_NOFS/wdstab6-27.ecsv'


# wdstab = Table.read(path, header_start=0, data_start=1)


# rownum = 100

# #read in the coordinates of the primary and secondary in WDS for the designated row number
# ra1, dec1 = wdstab['RApri-prepped'][rownum], wdstab['DECpri-prepped'][rownum]
# ra2, dec2 = wdstab['RAsec-prepped'][rownum], wdstab['DECsec-prepped'][rownum]
# # radius is in degrees
# radius = 5*u.arcsec
# coord1 = SkyCoord(ra=ra1 , dec = dec1, unit='deg')
# myquery1 = query_gaia(coordinate=coord1, radius=radius)

# radius = 5*u.arcsec
# coord2 = SkyCoord(ra=ra2 , dec = dec2, unit='deg')
# myquery2 = query_gaia(coordinate=coord2, radius=radius)

# # view the query results stacked together (there may be repeated objects if they are found by both queries)
# vstack([myquery1, myquery2])


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


source_id,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,radial_velocity,radial_velocity_error,astrometric_params_solved,visibility_periods_used,astrometric_sigma5d_max,ruwe,phot_g_mean_mag,phot_g_mean_flux_over_error,phot_bp_mean_mag,phot_bp_mean_flux_over_error,phot_rp_mean_mag,phot_rp_mean_flux_over_error,bp_rp,phot_bp_rp_excess_factor
,yr,deg,mas,deg,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,km / s,km / s,,,mas,,mag,,mag,,mag,,mag,
int64,float64,float64,float32,float64,float32,float64,float32,float32,float64,float32,float64,float32,float32,float32,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
2853967083422949376,2016.0,0.2512141918308061,0.025680063,27.342194266301533,0.0181289,2.5741498767455107,0.036708534,70.124016,-2.7785969702634845,0.04072014,-3.338309513371836,0.021903457,-2.7360594,0.6950137,31,19,0.05704284,1.7701749,11.480003,3489.204,11.716386,2920.0637,11.089971,4110.2344,0.62641525,1.1862055
2853967083422949120,2016.0,0.25114862193667487,0.011582456,27.344667880369762,0.008197574,4.207421653342171,0.017068105,246.50784,38.029028322933655,0.01814615,-54.802016629473336,0.009611096,-65.94423,1.6252601,31,19,0.025434896,1.0168203,13.611754,7126.7603,14.25992,1187.6969,12.841515,2024.096,1.4184055,1.2548844


## wds_in_gaia_query(core_num, total_cores) --- query WDS entries in Gaia and save results in a table


- The WDS is split up between the number of cores available, and we call this function for each separate instance in the multiprocessing.

- It goes through a portion of the WDS, makes a query around the WDS targets, and saves the results in an output table 

In [ ]:
def wds_in_gaia_query(core_num, total_cores): # core num starts at 0

    # this function queries gaia for the objects within a 5 arcsec radius of the component coordinates in the WDS

    #notes: there are commented out checkpoints used to troubleshoot this function

    ## vayu's lab comp
    # path = 'C:/Users/sc36/Documents/DaphneUSNO/NOFS copy-20230218T215456Z-001/NOFS copy/wdstab6-27.ecsv'

    save_path = '/home/student/djz7128/djz_NOFS/QueryResults'

    # wiser's lab comp
    path = '/home/student/djz7128/djz_NOFS/wdstab6-27.ecsv'



    wdstab = Table.read(path, header_start=0, data_start=1)

    # total number of queries will be the number of wds entries that we look at
    total_num_queries = len(wdstab)

    # find approx # of queries per core... ignoring the fraction
    queries_per_core = total_num_queries // total_cores
    leftover_rows = total_num_queries % total_cores


    # make a list of the start and end row variables
    start_row_list = []
    end_row_list = []

    # make a list to get the start and end row for each process (core)
    rownum_counter = 0
    for core in range(total_cores):
        start_row_list.append(rownum_counter)
        rownum_counter += queries_per_core
        if core == total_cores - 1:
            end_row_list.append(total_num_queries)
        else:
            end_row_list.append(rownum_counter)
    end_row_list[-1] = end_row_list[-1] + leftover_rows


    # define start and end row of wds for this specific process
    # start row is included in query, but end row is not included in the range
    wds_start_row =  start_row_list[core_num]
    wds_end_row = end_row_list[core_num]

    # All of the processes run simultaneously
    # This print statement updates the progress for the process
    print('core: ', core_num, 'end row = ', wds_end_row)

    # these are the column names that have a number data type...
    # the source ids need to stay as strings so I add those separately
    num_column_names = ['ref_epoch', 'ra', 'ra_error', 'dec',
                    'dec_error', 'parallax', 'parallax_error', 'parallax_over_error','pmra',
                    'pmra_error', 'pmdec', 'pmdec_error',
                    'radial_velocity', 'radial_velocity_error',
                    'astrometric_params_solved', 'visibility_periods_used',
                    'astrometric_sigma5d_max','ruwe',
                    'phot_g_mean_mag', 'phot_g_mean_flux_over_error',
                    'phot_bp_mean_mag', 'phot_bp_mean_flux_over_error',
                    'phot_rp_mean_mag', 'phot_rp_mean_flux_over_error',
                    'bp_rp','phot_bp_rp_excess_factor']





    # we will have a pair of stars for each row
    # this means we have two sets of query results in each row
    # put the parameters in a dictionary with suffixes _a and _b to name columns accordingly
    colname_dictionary = {}

    for column in num_column_names:
        colname_dictionary['{0}_a'.format(column)] = 0
        colname_dictionary['{0}_b'.format(column)] = 0

    # store all of the column names in colnames list
    colnames = []
    for entry in colname_dictionary:
        colnames.append(entry)




    """ BUILD OUTPUT TABLES """





    try:

        # if the tables have already been started, open them and continue from where they left off
        # this is a feature included because the code has a tendency to break before the process is completed
        # we already open qrt earlier so leave that one commented


        
        query_results_table = Table.read('{0}/query_results_table_c{1}.ecsv'.format(save_path, core_num), header_start=0, data_start=1)
        index_error_queries = Table.read('{0}/index_error_queries_c{1}.ecsv'.format(save_path, core_num), header_start=0, data_start=1)
        try:
            unknown_error_queries = Table.read('{0}/unknown_error_queries_c{1}.ecsv'.format(save_path, core_num), header_start=0, data_start=1)
        except:
            unknown_error_queries = Table(names = ('wds_identifier', 'wds_rownum'), dtype = ('a30', 'f8'))
            pass


        try:
            next_WDS_id = query_results_table['wds_identifier'][len(query_results_table)-1]
        except:
            next_WDS_id = 0
            pass

        for row in range(len(query_results_table)):
            try:
                if query_results_table[row] == next_WDS_id:
                    query_results_table.remove_row(row)
            except:
                pass

        for row in range(len(index_error_queries)-1):
            if index_error_queries['wds_identifier'][row] == next_WDS_id:
                index_error_queries.remove_row(row)

        for row in range(len(unknown_error_queries)-1):
            if unknown_error_queries['wds_identifier'][row] == next_WDS_id:
                unknown_error_queries.remove_row(row)

        query_results_table_rownum = max(len(query_results_table) - 1, 0)
        index_error_queries_rownum = max(len(index_error_queries) - 1, 0)
        unknown_error_queries_rownum = max(len(unknown_error_queries) - 1,0)

        # checkpoint
        print('previous tables read in')

        #initialize wds identifier
        wds_identifier = ''


    except:
            # the code above assumes that the tables exist... if they don't, we should make them

            next_WDS_id = 0

            # query results table will have all info for a pair of stars in one row
            query_results_table = Table(names=colnames)


            # add the wds identifier column and source id columns (doesn't work until I add one row to the table)
            query_results_table.add_row()
            query_results_table.add_column('                              ', name = 'wds_identifier', index = 0)
            query_results_table.add_column(00000000                        , name='wds_rownum', index = 1)
            query_results_table.add_column('                              ', name = 'source_id_a', index = 2)
            query_results_table.add_column('                              ', name = 'source_id_b', index = 3)



            # remove that first row -- the loop will add rows as needed
            query_results_table.remove_row(0)



            # index error wds info:
            index_error_queries = Table(names = ('wds_identifier', 'wds_rownum'), dtype = ('a30', 'f8'))

            # unknown error wds info:
            unknown_error_queries = Table(names = ('wds_identifier', 'wds_rownum'), dtype = ('a30', 'f8'))



            # initialize row numbers for each output table:
            query_results_table_rownum = 0
            index_error_queries_rownum = 0
            unknown_error_queries_rownum = 0





            #initialize wds identifier
            wds_identifier = ''

            print('new tables constructed')




    """
    LOOP THROUGH THE WDS 
    """

    # find starting rownum:
    # this will re-do a few queries to make sure they are complete
    new_start_row = int(query_results_table['wds_rownum'][-2])


    # now: go through the wds from the rows designated for the particular core and query those objects in Gaia
    for rownum in range(new_start_row, wds_end_row-1):


       # If the previous WDS identifier (from last iteration of loop) is the same is current one,
        # then this row was already accounted for in that previous query - so you can move on.
        # This is because there are some systems that have 3+ components: all of them share a WDS identifier,
        # and we query all components of a system a single iteration of the loop
        if wdstab['WDS Identifier'][rownum] == wds_identifier:
            pass

        else:
            # read in the wds identifier so we know which system is queried (this is how output files will connect to the input)
            wds_identifier = wdstab['WDS Identifier'][rownum]




        # if there are multiple columns with same WDS identifier,
        # query all of those objects and add them to gaiaresults list

        # this code starts at the current rownum, and keeps going as long as the wds_identifiers of the future rows are the same as the current one
        for shared_id_rownum in range(rownum, wds_end_row-1):
            if wdstab['WDS Identifier'][shared_id_rownum] == wds_identifier:

                print('\n core # ', core_num, 'of ', total_cores, 'cores   --- row number: ', rownum)
                """ make the 2 queries for given WDS row """


                # read in the coordinates for each of the WDS objects
                ra1, dec1, ra2, dec2 =wdstab['RApri-prepped'][rownum], wdstab['DECpri-prepped'][rownum], wdstab['RAsec-prepped'][rownum], wdstab['DECsec-prepped'][rownum]

                # query object 1
                radius1 = 5*u.arcsec
                coord = SkyCoord(ra=ra1 , dec = dec1, unit='deg')
                myquery1 = query_gaia(coordinate=coord, radius=radius1)

                # query object 2
                radius2 = 5*u.arcsec
                coord = SkyCoord(ra=ra2 , dec = dec2, unit='deg')
                myquery2 = query_gaia(coordinate=coord, radius=radius2)


                """ VERTICALLY STACK ALL QUERIES TO CREATE A LIST WITH ALL QUERIES FROM 1 WDS ROW """

                # first query for this WDS identifier: just add query 1 and 2 to list

                if len(myquery1) + len(myquery2) == 0:
                    index_error_queries.add_row()
                    index_error_queries['wds_identifier'][index_error_queries_rownum] = wds_identifier
                    index_error_queries['wds_rownum'][index_error_queries_rownum] = rownum
                    index_error_queries_rownum +=1

                    # checkpoint
                    # print('index error table updated')
                    pass
                elif shared_id_rownum == rownum:
                    gaiaresults = vstack([myquery1, myquery2])

                # then, keep adding the new queries to the existing gaiaresults list
                else:
                    gaiaresults = vstack([gaiaresults, myquery1, myquery2])


            # if WDS identifiers don't match, we have queried all component coordinates for the system -> move on
            else:
                pass


        try:

            """ REMOVE DUPLICATES FROM GAIA RESULTS TABLE """

            # checkpoint
            # print('length of gaiaresults is', len(gaiaresults))

            gaiaresults = unique(gaiaresults, keep = 'first', silent = 'True')

            # checkpoint
            # print('duplicates_removed')
            # print('length of gaiaresults is', len(gaiaresults))


            # save all query results where less than two unique objects are found to index error query table
            # if there's not at least two stars found, we can't do the analysis for that WDS system
            if len(gaiaresults) <= 1:
                index_error_queries.add_row()
                index_error_queries['wds_identifier'][index_error_queries_rownum] = wds_identifier
                index_error_queries['wds_rownum'][index_error_queries_rownum] = rownum
                index_error_queries_rownum +=1

                # update output table
                ascii.write(index_error_queries, '{path}/index_error_queries_c{core}.ecsv'.format(path = save_path, core = core_num), format='ecsv',overwrite=True)

                # checkpoint
                # print('index error table updated')
                pass


            else: # if there's more than two objects found -> keep going!


                """ CROSS CHECK EACH ENTRY WITH EACH OTHER """
                # avoid repeat comparisons

                # make a list of every unique combination of two objects in my list
                # this will be a comma separate string of source ids from Gaia
                L = gaiaresults['source_id']
                combolist = [",".join(map(str, comb)) for comb in combinations(L, 2)]

                # checkpoint
                # print('cross check complete')


                #make source id column the index for gaiaresults table
                # this allows us to return a row by searching the source id
                gaiaresults.add_index('source_id')


                # use the list of unique combinations and find both of those rows
                # then, compare them



                for combination in combolist:
                # every unique combination of the gaiaresults is cross-compared 
                # to see if any pair of stars found around the WDS coordinates are gravitationally associated

                    # the combination is a comma separated entry of two source ids -- unique combo
                    # then, split them up so I can call to the data about each specific target in the combo
                    # the source id is the index for my gaiaresults table, so I can call to the target row using the id
                    query_a, query_b = combination.split(',')
                    row_a = gaiaresults.loc[int(query_a)]
                    row_b = gaiaresults.loc[int(query_b)]

                    # checkpoint
                    # print('components assigned')
                    # print(gaiaresults)

                    """ READ IN THE RELEVANT INFO (source id and parallax): """

                    # read in the parameters for object a and b
                    # put the parameters in a dictionary with suffixes _a and _b accordingly
                    parameter_dictionary = {}

                    # go through all of the columns for both component a and b for each unique combo of gaiaresults
                    # and update the output tables with the queried information as well as the corresponding WDS identifier

                    # before updating the table, organize the information in a dictionary
                    for column in query_results_table.colnames:
                        if column == 'wds_identifier':
                            parameter_dictionary['wds_identifier'] = wdstab[rownum]['WDS Identifier']
                        # elif column == 'wds_rownum':
                        #     parameter_dictionary['wds_rownum'] == rownum

                        elif column.endswith('_a') == True:
                            param_len = len(column)
                            parameter_dictionary['{0}'.format(str(column))] = row_a[column[:param_len - 2]]
                        elif column.endswith('_b') == True:
                            param_len = len(column)
                            parameter_dictionary['{0}'.format(str(column))] = row_b[column[:param_len - 2]]


                    # make the next row for the query results output table
                    query_results_table.add_row()

                    # update the query results table with the info stored in the dictionary
                    for entry in parameter_dictionary:
                        query_results_table[entry][query_results_table_rownum] = parameter_dictionary[entry]

                    # update wds_rownum separately because it wasn't working in the loop
                    query_results_table['wds_rownum'][query_results_table_rownum] = rownum

                    query_results_table_rownum +=1

                    # update output file
                    ascii.write(query_results_table, '{path}/query_results_table_c{core}.ecsv'.format(path = save_path, core = core_num), format='ecsv',overwrite=True)

                    # checkpoint
                    # print('query_results_table updated')




        except:
            # if an unexpected error occurs, add to table of objects with any error:
            unknown_error_queries.add_row()
            unknown_error_queries['wds_identifier'][unknown_error_queries_rownum] = wds_identifier
            unknown_error_queries['wds_rownum'][unknown_error_queries_rownum] = rownum
            unknown_error_queries_rownum +=1

            # update output file
            ascii.write(unknown_error_queries, '{path}/unknown_error_queries_c{core}.ecsv'.format(path = save_path, core = core_num), format='ecsv',overwrite=True)

            #checkpoint
            # print('unknown error')

            pass



    ## vayu's lab comp:
    # save_path = 'C:/Users/sc36/Documents/DaphneUSNO/NOFS copy-20230218T215456Z-001/NOFS copy/QueryResults'

    # wiser's lab comp:
    save_path = '/home/student/djz7128/djz_NOFS/QueryResults'

    # write the output files to end in _c# where # is the core number that was used
    ascii.write(query_results_table, '{path}/query_results_table_c{core}.ecsv'.format(path = save_path, core = core_num), format='ecsv',overwrite=True)
    ascii.write(query_results_table, '{path}/query_results_table_c{core}.csv'.format(path = save_path, core = core_num), format='csv',overwrite=True)

    ascii.write(index_error_queries, '{path}/index_error_queries_c{core}.ecsv'.format(path = save_path, core = core_num), format='ecsv',overwrite=True)
    ascii.write(index_error_queries, '{path}/index_error_queries_c{core}.csv'.format(path = save_path, core = core_num), format='csv',overwrite=True)

    ascii.write(unknown_error_queries, '{path}/unknown_error_queries_c{core}.ecsv'.format(path = save_path, core = core_num), format='ecsv',overwrite=True)
    ascii.write(unknown_error_queries, '{path}/unknown_error_queries_c{core}.csv'.format(path = save_path, core = core_num), format='csv',overwrite=True)


In [ ]:
# sample tests... if you make the num_cores very high, then there's only a few queries for a process

# wds_in_gaia_query(1,2)
# wds_in_gaia_query(1,30000)
# wds_in_gaia_query(2,30000)
# wds_in_gaia_query(3,30000)



In [ ]:
len(wdstab)

154513


### Dividing up the WDS for multiprocessing
##### this is incorporated in the main wds_in_gaia_query function, just rewritten here for checking this component of the function

In [ ]:
# # Prepare for multiprocessing
# total_cores = 2

# # total number of queries will be the number of wds entries that we look at
# total_num_queries = len(wdstab)

# # find approx # of queries per core... ignoring the fraction
# queries_per_core = total_num_queries // total_cores
# leftover_rows = total_num_queries % total_cores

# # make a list of the start and end row variables0
# start_row_list = []
# end_row_list = []

# # make a list to get the start and end row for each process
# rownum_counter = 0
# for core in range(total_cores):
#     start_row_list.append(rownum_counter)
#     rownum_counter += queries_per_core
#     if core == total_cores - 1:
#         end_row_list.append(total_num_queries)
#     else:
#         end_row_list.append(rownum_counter)

# end_row_list[-1] = end_row_list[-1] + leftover_rows




## Initiate Gaia Query with multiprocessing

In [ ]:
def initiate_gaia_query(total_cores):

    # use multiprocsesing to query all objects in the WDS by splitting up the rows into separate processes that run concurrently


    processes=[]

    for core_num in range(total_cores):
        print('process initiated: core', core_num)
        p = multiprocessing.Process(target = wds_in_gaia_query, args = (core_num, total_cores))
        p.start()
        processes.append(p)

    for p in processes:
        p.join()


In [ ]:
#started back at 7:18 on 3/30
# again at 9:10
# agaian at 12:00
#again at 9:51 3/31
initiate_gaia_query(12)

process initiated: core 0
process initiated: core 1
process initiated: core 2
process initiated: core 3
process initiated: core 4
process initiated: core 5
process initiated: core 6
process initiated: core 7
process initiated: core 8
process initiated: core 9
process initiated: core 10
process initiated: core 11
core:  4 end row = core:  11  64380
end row = core:  3 core:  154514end row =  51504
 
9 end row = core:   1287607
 end row =  103008
core:  1core:  core: end row =    10 end row = 257520
 end row =   14163612876core: 

core:  2 end row =   38628
8 end row =  core:  6 end row = 115884 
90132
core:  5 end row =  77256
previous tables read in

 core #  3 previous tables read in
of 
 core #   12 9 cores   --- row number:  51501
of  12 cores   --- row number:  128757
previous tables read in

 core #  previous tables read in7
 of 
 core #  1 12 of  cores   --- row number:   103006
12 cores   --- row number:  25750
previous tables read in

 core #  11 of  12 cores   --- row number:  

/tmp/ipykernel_1207752/2541637854.py:369: StringTruncateWarning: truncated right side string(s) longer than 1 character(s) during assignment
  unknown_error_queries['wds_identifier'][unknown_error_queries_rownum] = wds_identifier


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


/home/student/djz7128/.conda/envs/djzconda/lib/python3.11/site-packages/astropy/table/column.py:1280: UserWarning: Warning: converting a masked element to nan.
  self.data[index] = value
/home/student/djz7128/.conda/envs/djzconda/lib/python3.11/site-packages/astropy/table/column.py:1280: UserWarning: Warning: converting a masked element to nan.
  self.data[index] = value


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]INFO: Query finished. [astroquery.utils.tap.core]

INFO: Query finished. [astroquery.utils.tap.core]

 core #  

/home/student/djz7128/.conda/envs/djzconda/lib/python3.11/site-packages/astropy/table/column.py:1280: UserWarning: Warning: converting a masked element to nan.
  self.data[index] = value


3 of 
 core #  12  9cores   --- row number:  of   12 51502
cores   --- row number:  128758


/home/student/djz7128/.conda/envs/djzconda/lib/python3.11/site-packages/astropy/table/column.py:1280: UserWarning: Warning: converting a masked element to nan.
  self.data[index] = value



 core #  2 of  12 cores   --- row number:  38626
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


## Test query multiprocessing

In [ ]:
# def initiate_test_gaia_query():


#     processes=[]

#     # num_of_processes is the number of cores to use
#     # divide_wds is a way to limit how many queries are done per core
#     # there are around 150,000 wds entries,
#     # so 150000/ divide_wds will be the number of queries per core

#     num_of_processes = 4
#     divide_wds = 300


#     for core_num in range(num_of_processes):
#         print('process initiated: core', core_num)
#         p = multiprocessing.Process(target = wds_in_gaia_query, args = (core_num, divide_wds))
#         p.start()
#         processes.append(p)


#     for p in processes:
#         p.join()



## Figure out number of available cores in the comp

In [ ]:
multiprocessing. cpu_count()

## Open the files that were saved in multiprocessing, save them to a single stacked table and a single stacked file

In [ ]:
total_cores = 12


file_dictionary = {}

for core_num in range(total_cores):

            file_dictionary['query_results_table_c{0}'.format(core_num)] = 0
            file_dictionary['index_error_queries_c{0}'.format(core_num)] = 0



## vayu's lab comp:
# save_path = 'C:/Users/sc36/Documents/DaphneUSNO/NOFS copy-20230218T215456Z-001/NOFS copy/QueryResults'

# wiser's lab comp:
save_path = '/home/student/djz7128/djz_NOFS/QueryResults'


for file in file_dictionary:

    file_dictionary[file] = Table.read('{0}/{1}.ecsv'.format(save_path, file), header_start=0, data_start=1)





# vertically stack all 20 sections of each table


query_results_table_list = []
index_error_queries_list = []


for file in file_dictionary:
    if file.startswith('query_results_table_c'):
        query_results_table_list.append(file_dictionary[file])
    elif file.startswith('index_error_queries_c'):
        index_error_queries_list.append(file_dictionary[file])



stack_query_results_table = vstack(query_results_table_list)
stack_index_error_queries = vstack(index_error_queries_list)


ascii.write(stack_query_results_table, '{0}/stack_query_results_table.ecsv'.format(save_path), format='ecsv')
ascii.write(stack_query_results_table, '{0}/stack_query_results_table.csv'.format(save_path), format='csv')


ascii.write(stack_index_error_queries, '{0}/stack_index_error_queries.ecsv'.format(save_path), format='ecsv')
ascii.write(stack_index_error_queries, '{0}/stack_index_error_queries.csv'.format(save_path), format='csv')


qrt ='{0}/stack_query_results_table.ecsv'.format(save_path)
ie = '{0}/stack_index_error_queries.ecsv'.format(save_path)
stack_query_results_table = Table.read(qrt, header_start=0, data_start=1)
stack_index_error_queries = Table.read(ie, header_start=0, data_start=1)



In [ ]:
stack_query_results_table

wds_identifier,wds_rownum,source_id_a,source_id_b,ref_epoch_a,ref_epoch_b,ra_a,ra_b,ra_error_a,ra_error_b,dec_a,dec_b,dec_error_a,dec_error_b,parallax_a,parallax_b,parallax_error_a,parallax_error_b,parallax_over_error_a,parallax_over_error_b,pmra_a,pmra_b,pmra_error_a,pmra_error_b,pmdec_a,pmdec_b,pmdec_error_a,pmdec_error_b,radial_velocity_a,radial_velocity_b,radial_velocity_error_a,radial_velocity_error_b,astrometric_params_solved_a,astrometric_params_solved_b,visibility_periods_used_a,visibility_periods_used_b,astrometric_sigma5d_max_a,astrometric_sigma5d_max_b,ruwe_a,ruwe_b,phot_g_mean_mag_a,phot_g_mean_mag_b,phot_g_mean_flux_over_error_a,phot_g_mean_flux_over_error_b,phot_bp_mean_mag_a,phot_bp_mean_mag_b,phot_bp_mean_flux_over_error_a,phot_bp_mean_flux_over_error_b,phot_rp_mean_mag_a,phot_rp_mean_mag_b,phot_rp_mean_flux_over_error_a,phot_rp_mean_flux_over_error_b,bp_rp_a,bp_rp_b,phot_bp_rp_excess_factor_a,phot_bp_rp_excess_factor_b
str13,int64,str19,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
00000+3852BU,3,2881742976228918912,2881742980523997824,2016.0,2016.0,0.007304252161929567,0.005058967400198452,0.016427384689450264,0.02496100589632988,38.85872352417506,38.859269115054886,0.014054271392524242,0.019697383046150208,2.955704163346745,2.992686610903894,0.027848364785313606,0.04275492951273918,106.1356430053711,69.99629211425781,5.04121006557981,5.761322176559112,0.01744064688682556,0.027374057099223137,-2.2337674557458724,-2.4055595285991993,0.015975115820765495,0.022997289896011353,-5.568790435791016,nan,0.7653659582138062,nan,31.0,31.0,20.0,18.0,0.028870442882180214,0.046229343861341476,1.2354885339736938,1.0191636085510254,11.427901268005371,6.596024990081787,1726.3349609375,2948.464111328125,11.674772262573242,6.584270477294922,90.04275512695312,1470.9251708984375,10.888983726501465,6.558365345001221,262.5050964355469,930.067626953125,0.7857885360717773,0.025905132293701172,1.2692034244537354,1.16890287399292
00001+7727LOC,6,540288988710861824,540288988710861952,2016.0,2016.0,0.025039271133275212,0.03472491134344492,0.050575174391269684,0.013834286481142044,77.44280871485321,77.44362726856326,0.04344682767987251,0.01188855990767479,6.7716161348462895,6.894360456576272,0.05322949215769768,0.014487730339169502,127.21549224853516,475.8758239746094,85.8194419212781,85.46432115647032,0.06538749486207962,0.017955169081687927,36.5657437504867,36.795972656600284,0.05974858999252319,0.016469508409500122,nan,-27.13381004333496,nan,8.205977439880371,31.0,31.0,25.0,25.0,0.09252406656742096,0.025319892913103104,1.21744704246521,1.072059154510498,16.57539176940918,14.10311508178711,904.2947387695312,1744.7960205078125,18.147706985473633,15.268089294433594,80.51824951171875,421.7792663574219,15.334451675415039,13.029826164245605,532.9720458984375,699.989501953125,2.8132553100585938,2.2382631301879883,1.4904694557189941,1.3791841268539429
00001+5400ES,7,396305497218596096,396305570238406144,2016.0,2016.0,0.030011439476742407,0.028112788664484196,0.008326414041221142,0.015584493987262249,53.999576973903956,54.00012486314701,0.008348978124558926,0.014539013616740704,2.384840253174304,2.3315141654599225,0.013162785209715366,0.023133113980293274,181.18052673339844,100.78687286376953,-0.9147508384598348,-1.064158531848239,0.010332660749554634,0.01920725405216217,-8.803328013644308,-8.914049677675674,0.011068823747336864,0.01896110363304615,-3.078555107116699,0.7382858991622925,1.3963912725448608,1.2604676485061646,31.0,95.0,26.0,26.0,0.017494510859251022,0.030793266370892525,0.93769371509552,1.0648890733718872,12.610419273376465,10.702967643737793,5557.79931640625,30

In [ ]:
stack_index_error_queries

wds_identifier,wds_rownum
str13,float64
00000+7530A,0.0
00000+4004ES,1.0
00000+0044SKF,4.0
00000-0530OCC,5.0
00001+0638SLW,10.0
00001+0022SKF,11.0
...,...
23599-3112TDT,154512.0
23599-3112TDT,154512.0
